<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2019</h1>
<h1 style="font-size: 250%;">Midterm Exam</h1>
<h1>Thursday March 7, 2019</h1>
</div>

**Instructions:** Answer all questions using the cells provided. Feel free to add additional cells to your notebook, but there is no guarantee that they will be considered when grading. If any question seems unclear or appears to lack sufficient information, state a reasonable assumption in the cell provided for your answer and proceed.

You can use any notes, books, or on-line materials to aid when answering your questions, so long as they do not involve communication with any other person.

---
The following three problems refer to the following 1000-base dna sequence:

In [ ]:
seq = "CGGTTCGATCCCGCTAGGCTCCATCAGGATACAATCCTACTAAACTTAATACAAGTGAAGTTGAACACGCAACTCACTTCCTAGGAAAATGGACAATCTT" \
    + "CCCTTGTGTGCAAGGCCCACATGGTCAGATTCCTAATTTTCTACAGAAGTTTCGCTAAAGCGAGCGTTGCTTAGTATCCTATATAATAGTCCATTGAAAA" \
    + "TTGAATATCTATATCAAATTCCACGATCTAGAAATAGATTGTAGAAAAGTAACAAGAAAATAAACCGAAAACGCTGTGACTATTTAATAAGTTTTCTAGT" \
    + "TTAAAAAAACTAGGTTAATAAGGTTAAGTTAATAAGGGCGCACGGTGGATGCCTTGGCACTAGAAGCCGAAGAAGGACGTGACTAACGACGAAATGCTTT" \
    + "GGGGAGCTGTAAGTAAGCGCTGATCCAGAGATGTCCGAATGGGGGAACCCGGCATGTAATGCATGTCATCCATGACTGTTAAGGTCATGAGAAGGAAGAC" \
    + "GCAGTGAACTGAAACATCTAAGTAGCTGCAGGAAGAGAAAGCAAACGCGATTGCCTTAGTAGCGGCGAGCGAAACGGCAGGAGGGCAAACCGAGGAGTTT" \
    + "ACTCCTCGGGGTTGTAGGACTGCGAAGTGGGACATAAAGTTAATAGAAGAATTACCTGGGAAGGTAAGCCAAAGAGAGTAACAGCCTCGTATTTAAAATT" \
    + "GACTTTAGCCCTAGCAGTATCCTGAGTACGGCGAGACACGCGAAATCTCGTCGGAATCTGGGAGGACCATCTCCCAACCCTAAATACTCTCTAGTGACCG" \
    + "ATAGTGAACCAGTACCGTGAGGGAAAGGTGAAAAGCACCCCGGGAGGGGAGTGAAATAGAACCTGAAACCGTGTGCCTACAACAAGTTCGAGCCCGTTAA" \
    + "TGGGTGAGAGCGTGCCTTTTGTAGAATGAACCGGCGAGTTACGATATGATGCGAGGTTAAGTTGAAGAGACGGAGCCGTAGGGAAACCGAGTCTTAATAG"



In [ ]:
# seq = seq[0]
def kmerCounts(seq, k):
    kmerDict = {}
    for i in range(1,len(seq)-k+1):
        kmer = seq[i:i+k]
        kmerDict[kmer] = kmerDict.get(kmer,0) + 1
    return kmerDict

print('  k     k-mers              4^k      N-k+1          missing   repeated')
for k in range(2,25):
    kmers = kmerCounts(seq, k)
    print("%3d %10d %16d %10d %16d %10d" % (k, len(kmers), 4**k, (len(seq)-1)-k+1, 4**k-len(kmers), (len(seq)-1)-k+1-len(kmers)))
    
kmers = kmerCounts(seq, 4)
    
def printAllKLength(set, k): 
    n = len(set)  
    printAllKLengthRec(set, "", n, k) 
  
def printAllKLengthRec(set, prefix, n, k): 
      
    # Base case: k is 0, 
    # print prefix 
    if (k == 0) : 
#         print(prefix)
        all_kmers.append(prefix)
        return
  
    # One by one add all characters  
    # from set and recursively  
    # call for k equals to k-1 
    for i in range(n): 
  
        # Next character of input added 
        newPrefix = prefix + set[i] 
          
        # k is decreased, because  
        # we have added a new character 
        printAllKLengthRec(set, newPrefix, n, k - 1)
all_kmers = []
set1 = ['A','C','G','T']
printAllKLength(set1, 4)
# print(all_kmers)
print("\n\n4 missing kmers:")
print(list(set(all_kmers) - set(kmers)))

---
**Problem #1:** What is the smallest value of *k* where *seq* does not contain every possible *k*-mer? Give a list of all missing *k*-mers for that value of k.

---
**Problem #2:** What is the smallest value of *k* where every *k*-mer in *seq* is distinct (i.e. it contains no repeated *k*-mers)?

---
**Problem #3:** Suppose that you constructed a De Bruijn graph using all *k*-mers from *seq*, with *k* set to one larger than your solution to **Problem #2**. What would be the maximum in-degree and out-degree for any vertex in this graph? Explain your answer?

In [ ]:
class Graph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}
        self.vertex = {i:v for i,v in enumerate(vlist)}
        self.edge = []
        self.edgelabel = []
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
    def hamiltonianPath(self):
        """ A Brute-force method for finding a Hamiltonian Path. 
        Basically, all possible N! paths are enumerated and checked
        for edges. Since edges can be reused there are no distictions
        made for *which* version of a repeated edge. """
        for path in itertools.permutations(sorted(self.index.values())):
            for i in xrange(len(path)-1):
                if ((path[i],path[i+1]) not in self.edge):
                    break
            else:
                return [self.vertex[i] for i in path]
        return []
    def SearchTree(self, path, verticesLeft):
        """ A recursive Branch-and-Bound Hamiltonian Path search. 
        Paths are extended one node at a time using only available
        edges from the graph. """
        if (len(verticesLeft) == 0):
            self.PathV2result = [self.vertex[i] for i in path]
            return True
        for v in verticesLeft:
            if (len(path) == 0) or ((path[-1],v) in self.edge):
                if self.SearchTree(path+[v], [r for r in verticesLeft if r != v]):
                    return True
        return False
    def hamiltonianPathV2(self):
        """ A wrapper function for invoking the Branch-and-Bound 
        Hamiltonian Path search. """
        self.PathV2result = []
        self.SearchTree([],sorted(self.index.values()))                
        return self.PathV2result
    def degrees(self):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in self.edge:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return inDegree, outDegree
    def verifyAndGetStart(self):
        inDegree, outDegree = self.degrees()
        start = 0
        end = 0
        for vert in self.vertex.keys():
            ins = inDegree.get(vert,0)
            outs = outDegree.get(vert,0)
            if (ins == outs):
                continue
            elif (ins - outs == 1):
                end = vert
            elif (outs - ins == 1):
                start = vert
            else:
                start, end = -1, -1
                break
        if (start >= 0) and (end >= 0):
            return start
        else:
            return -1
    def eulerianPath(self):
        graph = [(src,dst) for src,dst in self.edge]
        currentVertex = self.verifyAndGetStart()
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                for edge in graph:
                    try:
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print("There is no path!")
                    return False
        return path
    def eulerEdges(self, path):
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeList = []
        for i in range(len(path)-1):
            edgeList.append(self.edgelabel[edgeId[path[i],path[i+1]].pop()])            
        return edgeList
    def render(self, highlightPath=[]):
        """ Outputs a version of the graph that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edgeId = {}
        for i in xrange(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeSet = set()
        for i in xrange(len(highlightPath)-1):
            src = self.index[highlightPath[i]]
            dst = self.index[highlightPath[i+1]]
            edgeSet.add(edgeId[src,dst].pop())
        result = ''
        result += 'digraph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for index, label in self.vertex.iteritems():
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (index, label)
        for i, e in enumerate(self.edge):
            src, dst = e
            result += '    N%d -> N%d' % (src, dst)
            label = self.edgelabel[i]
            if (len(label) > 0):
                if (i in edgeSet):
                    result += ' [label="%s", penwidth=3.0]' % (label)
                else:
                    result += ' [label="%s"]' % (label)
            elif (i in edgeSet):
                result += ' [penwidth=3.0]'                
            result += ';\n'                
        result += '    overlap=false;\n'
        result += '}\n'
        return result

---
In the following two problems assume you are given 100 sequences each 1000 base pairs long and asked to find a 10-base motif for a binding site of a transcription-factor known to interact with each of the given sequences.

**Problem #4:** Suppose you are told that no binding site can differ in more than 1 position from the optimal motif. How might this contraint impact your search strategy? 

---
**Problem #5** Suppose you are told to only consider motif's that are their own reverse complement. For example, the reverse complement of *GAGCATGCTC* is *GAGCATGCTC*. How would you incorporate this constraint into your search strategy?

---
The following problems refer to the following multi-string Burrows-Wheeler Transform:

In [ ]:
msBWT = "TCAATGATAATGCCTTTTTTTTTTTAGGGTGAGAGGGGAAAAAAAAATACAAAATAACACAGCCCACACCCCTCCCAAAAAAAAAAATTAAATCATACAA" \
      + "AATATAAGAATTTATAAGGATATCAAAAAATGCGAAAATAAGATTTTGAAGCAAAAGAAGAGTGAATGAAATTGAGGCCAAAGGAGAAACGACTCATTTT" \
      + "GGGCCCCTGCGGGGGGAAGGGTTCTTTCCCTGTGGACAGGCTTAAAGGGTTATACATAGCTAAGACGTACGAGTGGAATCAAGGGTAAACTAAAAACGAA" \
      + "AATGAAAATAGAAGACCCAGATGAAGTATAATTTTGTAAAAAAAGAGGATGAGTAAAAGAATTGTATGGGATAAGCCCCGGGAGGAGCCCTCTTTAAAAG" \
      + "ACAAGAATCTAACGGTGGACGAAACTGCAATCGAGGAAAGGACAGTCCCAAAACACTCTTAAAGAGCCTCTAAATGATTTTGGAACAAGGAAAAAAATTG" \
      + "GCAAAGGGAAATTCGCAAATAAATAACATATAAAAGGAAAGAATGATGAAAAAAACAAATCAATATCAGGTAAGCTAAGGAAAAAGAAAGTAGGTGATTG" \
      + "ACAGCGCCCTGGGGACAAGGAAATAAAAAGTTAAAGGAGAGTGGAAATGGGGAAGGGGTGAGTTGGTAAAGAATACTACAAAAAACTATATACCCGGTTT" \
      + "TGCCAAAAGGTAGATATCGCAACGGAAAAAACGGAGACGATGAAAGCATGCCATTCGTCTAAGGGAAAAAGGGAACCAAACTAAATACATAATAGGGAAA" \
      + "AGATCGAAGCATCAAAAGAATTGATAATATTTAAAATTGGAAAAACGAAAATAATTGTTTAATTTGTTGTGTAACACTTTTTTTAAGGCAACAATGACTT" \
      + "AGGAATAAACATAAACAGATAGATTAAAAATTTCTCCTATGTAAAGGGTTTTA$AAAGAGTAATGACAGAATAAATGGAGGGAAATAGTCCG$ATCACAA" \
      + "AAGTATAAAAA$A$AGAGAAGAAATGACATCGGGCAA$AAAGAGGGGCGAAAATTTCATCACGAAAACAAGAGGTAGAAGATTGACATAAAAAAAAACAT" \
      + "GACCTATTTTTTTTAATTTAAACTGAAATTTTTTAACTGGCTTTTTTTATCCAAAGTCCAATGATGTTTGTAAATACTTCGTGACCGTTTCGTCCCCAAA" \
      + "ATATTTAAATATACGGGTCTGCGACGGGATCCACAGAGTATATACTAACAATACAGATTTGAAAAAATATCAAACAGAATACCCCCAAAATGCGTTTGAG" \
      + "TTAGTTCAAGAACCACCTTCAGATGCAAGCTGGGAACCAGGCCG$TCGGGGAACGGGAATGGTGAAGGGG$$GACAATTATCGGCCCACTTTCCGTGGCA" \
      + "TGAGGACCCTAG"
print("Length of string:" ,len(msBWT))
print("Num $s:",msBWT.count("$"))


In [ ]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].items()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

# def mergeBWT(bwt1, bwt2):
#     interleave = [(c, 0) for c in bwt1] + [(c, 1) for c in bwt2]
#     passes = min(len(bwt1), len(bwt2))
#     for p in range(passes):
#         i, j = 0, 0
#         nextInterleave = []
#         for c, k in sorted(interleave, key=lambda x: x[0]):
#             if (k == 0):
#                 b = bwt1[i]
#                 i += 1
#             else:
#                 b = bwt2[j]
#                 j += 1
#             nextInterleave.append((b, k))
#         if (nextInterleave == interleave):
#             break
#         interleave = nextInterleave
#     return ''.join([c for c, k in interleave])
def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

# bwt1 = "TG$TC$AAAA"
# bwt2 = "AAGTGTA$A$"
# bwt12 = mergeBWT(bwt1, bwt2)
# print(bwt12)
FM, Offset = FMIndex(msBWT)
# print(len(bwt12))
print("Num Strings:",round(1412/178))
for i in range(len(msBWT)):
    recover = recoverSuffix(i,msBWT,FM,Offset)
    print(i, ": ", recover, len(recover))
#     j = (i>>2)+(i&3)*(len(bwt12)//4)
#     print("%2d: %s" % (j, recoverSuffix(j,msBWT,FM,Offset)))


---
**Problem #6.** How many strings are encoded in this msBWT and are all strings of a uniform length? Explain how you arrived at your answers.

---
**Problem #7.** How many times does the substring *"TAAAAAC"* appear in the strings of the given msBWT? Does *"TAAAAAC"* appear in all of the encoded strings? Explain how you arrived at your answers.

In [ ]:
def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi
print("FM:",FM)
print("Offset:",Offset)
print("FM[1]:", FM[4])
# print(len(FM))

def findFirst(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:] < pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

def findLast(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:suffixarray[middle]+len(pattern)] <= pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

print(findBWT("TAAAAAC", FM, Offset))
last = findLast("an", t, sa)
print(first, last)
for suffix in sa[first:last]:     # recall "first" was found on the previous slide
    print("%3d: %s" % (suffix, t[suffix:]))
print(last - first, "times")

---
**Problem #8.** In an uncompressed msBWT each character is the predecessor of the suffix array of the encoded set of strings. *Describe an algorithm* to find the *sucessor* character of each string of the suffix array (the second letter of each suffix). You can assume that a full FMindex and offset list are available. *Estimate the performance of your algorithm*.

---
**Problem #9**. How well does the successor character of each string from a suffix array compress in comparision to the msBWT? Does the first-last property apply to this string of sucessor characters? Explain your answers.